In [1]:
import numpy as np

# Initialize inputs
book_balance_total = 100_000
cd_terms = np.zeros(121)  # 121 terms including 0 term
origination_points = np.array([1, 4, 6, 13, 77, 54])  # Points where balances are originated
distribution_percentages = np.array([0.1, 0.2, 0.15, 0.25, 0.1, 0.2])  # Must sum up to 1

# Verify that distribution percentages sum up to 1
if np.sum(distribution_percentages) != 1:
    raise ValueError("The sum of distribution percentages must be equal to 1.")

# Distribute the initial book balance across the specified CD terms
for point, percentage in zip(origination_points, distribution_percentages):
    cd_terms[point] += percentage * book_balance_total

# Forecast for 36 periods
forecast_periods = 36
forecast = np.zeros(forecast_periods)

for period in range(forecast_periods):
    # Shift balances to the next term (simulate the passage of one period)
    cd_terms = np.roll(cd_terms, -1)
    cd_terms[-1] = 0  # Reset the last term to 0, as it has matured
    
    # Calculate the total book balance for the current period
    forecast[period] = np.sum(cd_terms)

# Print or return the forecast array
print(forecast)

[100000.  90000.  90000.  90000.  70000.  70000.  55000.  55000.  55000.
  55000.  55000.  55000.  55000.  30000.  30000.  30000.  30000.  30000.
  30000.  30000.  30000.  30000.  30000.  30000.  30000.  30000.  30000.
  30000.  30000.  30000.  30000.  30000.  30000.  30000.  30000.  30000.]


In [23]:
import numpy as np

forecast_periods = 36
cd_terms = 121

# Assuming these matrices are provided
origination_matrix = np.random.randint(1, 8, size=(forecast_periods, 6))  # Random for example
origination_matrix = np.array([np.random.choice(range(1, 8), size=6, replace=False) for _ in range(forecast_periods)])

distribution_matrix = np.random.dirichlet(np.ones(6), size=forecast_periods)  # Random distributions that sum to 1 for example
projected_book_totals = np.linspace(100_000, 200_000, forecast_periods)  # Linear growth from 100k to 200k for example






# Initialize the forecast matrix with zeros
forecast_matrix = np.zeros((forecast_periods, cd_terms)) 

for period in range(forecast_periods):
    # Adjust the book balance for the period to simulate different amounts
    book_balance_total = projected_book_totals[period]
    promos = origination_matrix[period]
    dist = distribution_matrix[period]
    # Distribute the book balance for the current period
    cd_terms = np.zeros(121)
    cd_terms[promos] =  dist
    cd_terms = cd_terms * book_balance_total
 
    # Save the distribution to the forecast matrix for the current period
    forecast_matrix[period] = cd_terms

# Test
pd.DataFrame({"projection": pd.DataFrame(forecast_matrix).sum(axis=1), 'actual': projected_book_totals})



,projection,actual
0,100000.000000,100000.000000
1,102857.142857,102857.142857
2,105714.285714,105714.285714
3,108571.428571,108571.428571
4,111428.571429,111428.571429
5,114285.714286,114285.714286
6,117142.857143,117142.857143
7,120000.000000,120000.000000
8,122857.142857,122857.142857
9,125714.285714,125714.285714


In [127]:
import numpy as np

def get_amounts(df):    
    amounts = pd.DataFrame({"<2%": [0, 0], 
                            "2-4.5%": [100_000, 200_000], 
                            ">5%": [1_000_000, 2_000_000]}, 
                           index=['Under 100k', 'Over 100k'])

    return amounts

def create_projected_book_totals(amounts, env_index):
    """
    Project book totals based on environmental conditions and amounts tied to various conditions.

    Parameters:
    - amounts (pandas.DataFrame): DataFrame with rows representing categories (e.g., 'Under 100K', 'Over 100K')
                                  and columns representing conditions (e.g., '<2%', '2-5%', '>5%'). Values in the
                                  DataFrame represent amounts associated with each category and condition.
    - env_index (numpy.array): Array of integers where each element corresponds to a specific condition in the
                               'amounts' DataFrame. Mapping is based on the order of columns in 'amounts', starting from 1.

    Process:
    1. Column Mapping: Create a mapping of env_index values to DataFrame columns.
    2. Projection Initialization: Initialize a dictionary to hold projected book totals for each category.
    3. Population of Projections: Iterate through env_index, select relevant column for each index, and append amounts
                                  to projected totals for each category.
    4. Conversion to Arrays: Convert lists in projected_book_totals to numpy arrays for numerical operations.
    5. Extracting Forecasts: Identify 'Under' and 'Over' indices, extract corresponding projections, ensuring order.

    Returns:
    - tuple: Contains two numpy arrays (under_projection, over_projection) representing the projected book totals for
             categories containing "Under" and "Over", respectively.
    """
    
    env_column_mapping = {(i+1):col for i, col in enumerate(amounts.columns)}
    # Initialize dictionaries to hold the projected book totals for 'Under' and 'Over'
    projected_book_totals = {bal_cat:[] for bal_cat  in amounts.index}
    
    # Iterate through the env_index to populate the projected_book_totals
    for index in env_index:
        column = env_column_mapping[index]
       
        for category in projected_book_totals.keys():
            
            projected_book_totals[category].append(amounts.loc[category, column])
    
    # Convert lists to numpy arrays for further processing
    for category in projected_book_totals.keys():
        projected_book_totals[category] = np.array(projected_book_totals[category])
    
    # Return the tuple with 'Under' and 'Over' forecasts
    
    under = amounts.index[amounts.index.str.contains("under", case=False)][0]
    over = amounts.index[amounts.index.str.contains("over", case=False)][0]
    return (projected_book_totals[under], projected_book_totals[over])

    

def NBV_projection(forecast_periods, cd_terms, origination_matrix, distribution_matrix, projected_book_totals):
    """
    Function to project the Net Book Value (NBV) across multiple periods and CD terms.

    Parameters:
    - forecast_periods (int): Number of periods to forecast.
    - cd_terms (int): Number of CD terms to consider.
    - origination_matrix (np.array): Matrix indicating origination points for each period.
    - distribution_matrix (np.array): Matrix of distribution percentages for each origination point and period.
    - projected_book_totals (np.array): Array of projected book totals for each period.

    Returns:
    - np.array: Forecast matrix with dimensions (forecast_periods, cd_terms).
    """

    # Initialize the forecast matrix with zeros
    forecast_matrix = np.zeros((forecast_periods, cd_terms))

    for period in range(forecast_periods):
        # Adjust the book balance for the period to simulate different amounts
        book_balance_total = projected_book_totals[period]
        promos = origination_matrix[period]
        dist = distribution_matrix[period]

        # Distribute the book balance for the current period
        cd_term_forecast = np.zeros(cd_terms)
        cd_term_forecast[promos] = dist * book_balance_total

        # Save the distribution to the forecast matrix for the current period
        forecast_matrix[period] = cd_term_forecast

    return forecast_matrix


# Example usage
forecast_periods = 36
cd_terms = 121


amounts = get_amounts(pd.DataFrame)

env_index = np.concatenate((np.repeat(1,10), np.repeat(2,10), np.repeat(3,16) ))   # Note: Changed the upper limit to 4 to include the possibility of '>5%'

origination_matrix = np.array([np.random.choice(range(1, 8), size=6, replace=False) for _ in range(forecast_periods)])
distribution_matrix = np.random.dirichlet(np.ones(6), size=forecast_periods)

under_forecast, over_forecast = create_projected_book_totals(amounts, env_index)
 


# projected_book_totals = np.linspace(100_000, 200_000, forecast_periods)

forecast_distributed = NBV_projection(forecast_periods, cd_terms, origination_matrix, distribution_matrix, under_forecast)
pd.DataFrame({"projection": pd.DataFrame(forecast_distributed).sum(axis=1), 'actual': under_forecast})
 

,projection,actual
0,0.0,0
1,0.0,0
2,0.0,0
3,0.0,0
4,0.0,0
5,0.0,0
6,0.0,0
7,0.0,0
8,0.0,0
9,0.0,0


In [118]:
    # projected_book_totals = {'Under': [], 'Over': []}

env_column_mapping = {(i+1):col for i, col in enumerate(amounts.columns)}
amounts.index[amounts.index.str.contains("under", case=False)]

Index(['Under 100k'], dtype='object')

In [119]:


# Example usage


In [93]:
env_index = np.concatenate((np.repeat(1,10), np.repeat(2,10), np.repeat(3,16) ))   # Note: Changed the upper limit to 4 to include the possibility of '>5%'

 


 
 
def custom_sort_key(item):
    # Check if 'Under' or 'Over' is in the item, and assign a sort value based on that'
    item = item.upper()
    if 'OVER' in item:
        primary_sort = 0  # 'Under' comes first
    elif 'UNDER' in item:
        primary_sort = 1  # 'Over' comes after 'Under'
    else:
        primary_sort = 2  # Fallback value, in case neither 'Under' nor 'Over' is found
    
 
    secondary_sort = item
    print(primary_sort, secondary_sort)
    return (primary_sort, secondary_sort)
 
amounts = pd.DataFrame({
    "<2%": [0, 0],
    "2-5%": [100_000, 200_000],
    ">5%": [1_000_000, 2_000_000]
}, index=['Over - <100K' ,'Under - <100K'])


# Sort the index using the custom sort key
sorted_index = sorted(amounts.index, key=custom_sort_key)

# Reindex the DataFrame using the sorted index
sorted_amounts = amounts.reindex(sorted_index)
sorted_amounts

0 OVER - <100K
1 UNDER - <100K


,<2%,2-5%,>5%
Over - <100K,0,100000,1000000
Under - <100K,0,200000,2000000


,<2%,2-5%,>5%
Over 100K,0,200000,2000000
Under 100K,0,100000,1000000


In [71]:
np.repeat(1,10) # np.repeat(2,10)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])